## Version Description
In this version, we deploy a Hybrid PINN to predict only one correction factor for the MTR ( `MTR-600` ) by the neural network.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
np.random.seed(13)

# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head()

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41
3,A4,36.383413,39.101836,38.110486,37.597728,36.190671,37.706898,36.668551,37.759174,38.497755,...,41,41,41,41,41,41,41,41,41,41
4,A5,36.897000,39.925000,38.704000,38.175000,40.591000,39.913000,37.641000,38.309000,38.600000,...,41,41,41,41,41,41,41,41,41,41


In [4]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 58 columns):
HoV               34 non-null object
CAOLH_C66-C68     34 non-null float64
CAOLH_C68-C70     34 non-null float64
CAOLH_C70-C72     34 non-null float64
CAOLH_C72-C74     34 non-null float64
CAOLH_C74-C76     34 non-null float64
CAOLH_C76-C78     34 non-null float64
CAORH_C66-C68     34 non-null float64
CAORH_C68-C70     34 non-null float64
CAORH_C70-C72     34 non-null float64
CAORH_C72-C74     34 non-null float64
CAORH_C74-C76     34 non-null float64
CAORH_C76-C78     34 non-null float64
LAOLH_C66-C68     34 non-null float64
LAOLH_C68-C70     34 non-null float64
LAOLH_C70-C72     34 non-null float64
LAOLH_C72-C74     34 non-null float64
LAOLH_C74-C76     34 non-null float64
LAOLH_C76-C78     34 non-null float64
LAORH_C66-C68     34 non-null float64
LAORH_C68-C70     34 non-null float64
LAORH_C70-C72     34 non-null float64
LAORH_C72-C74     34 non-null float64
LAORH_C74-C76     34 non

In [5]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [6]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

In [11]:
# LTR Dataframe col index for Restrictor Zeta and Area values
target_df.iloc[:,58:]

,R600_HD_Area,R600_HD_Zeta3D,R610_HS1_Area,R610_HS1_Zeta3D,R611_HS1_Area,R611_HS1_Zeta3D,R612_HS1_Area,R612_HS1_Zeta3D,R613_HS1_Area,R613_HS1_Zeta3D,...,R620_HS2_Area,R620_HS2_Zeta3D,R620_HS3_Area,R620_HS3_Zeta3D,R621_HS1_Area,R621_HS1_Zeta3D,R621_HS2_Area,R621_HS2_Zeta3D,R621_HS3_Area,R621_HS3_Zeta3D
0,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
1,0.017437,4.730821,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
2,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.008954,9.156844,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
3,0.018627,3.764504,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
4,0.017203,4.949375,0.007288,5.595443,0.007691,4.756868,0.010210,6.271347,0.010210,6.114035,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
5,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
6,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
7,0.018385,3.942093,0.006585,7.506119,0.007037,6.198696,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
8,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
9,0.017671,4.521189,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119


**Store Loss Coefficient ($\zeta$) Values and Names for all Restrictors:**

In [12]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [13]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [14]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
# LTR_zeta_df.tail()

In [15]:
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)

### Training data Preparation

In [16]:
from Hybrid_PINN_v1_1 import NormbyMax, DeNormbyMax, PINN

In [17]:
input_features = ['MIXP','Density(kg/m³)','R600_HD_Zeta3D','TZ6_Flow']
features_max, df_rescaled = NormbyMax(final_df,input_features)
print('Max value of all features:',features_max)

V_max_org = features_max['TZ6_Flow']
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'MIXP': 2632.9689, 'Density(kg/m³)': 1.238, 'R600_HD_Zeta3D': 5.4148802645163, 'TZ6_Flow': 764.0123719999998}
Maximum TZ6 Flow: 764.0123719999998


In [18]:
# Display original and Scaled DataFrame Side-by-side
# print( '{:<60s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
# display_side_by_side(final_df[['HoV']+input_features],df_rescaled)

In [19]:
df_rescaled_inv = DeNormbyMax(df_rescaled,features_max,input_features)

In [20]:
train_data = df_rescaled
# train_data = df_rescaled.iloc[[7,15]]
print("Number of training samples:", len(train_data))
train_data

Number of training samples: 34


,MIXP,Density(kg/m³),R600_HD_Zeta3D,TZ6_Flow
0,0.987478,0.986026,0.914032,0.966062
1,0.987478,0.987722,0.873670,0.965658
2,0.989830,0.976575,0.762192,0.969853
3,0.987440,0.999031,0.695215,0.958246
4,0.987478,1.000000,0.914032,0.970963
5,0.987478,0.985299,0.914032,0.973195
6,0.987478,0.978837,0.914032,0.968664
7,0.988183,0.967367,0.728011,0.971982
8,0.985737,0.960420,0.762192,0.968232
9,0.987478,0.996446,0.834956,0.968331


In [21]:
# Store indices and values of train data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = final_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [22]:
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((np.asarray(y_hat)-np.asarray(y_true))**2)

**Train the Neural Network**:

In [23]:
### Set the hyperparameters here ###
iterations = 250
learning_rate = 0.01 ### Important Parameter
hidden_nodes = 2 ### Important Parameter
output_nodes = 1
batch_sample_size = 3
MTR_epsi = np.float(MTR_cf * 0.01)

In [24]:
%%time
N_i = X_train.shape[1]
network = PINN(N_i, hidden_nodes, output_nodes, learning_rate, MTR_epsi)

MSE_flowloss_hist = []
err_loss_hist = []
c_f_hist = {'HoV':[],'c_f_hat':[],'FlowRate_Diff_(LTR-FDDN)':[],'ConvergedEpoch':[]}

for ii in range(iterations):
    ## minibatch Gradient Descent
    print("\nEPOCH {} - Length of Training Dataset:".format(ii),len(train_data_idx)) #Check available training data size at the start of every epoch
    batch_idx = np.random.choice(train_data_idx, replace = False, size = batch_sample_size) # train_data_idx
    X_train = train_data[input_features].loc[batch_idx].values
    dia_train = train_R600HD_series.loc[batch_idx]        
    print('Current Indices in Batch:',batch_idx) #'\nOld_Zeta:',list(target_df[[MTR+'_Zeta3D']].loc[batch_idx].values[0]))
    error,V_hat_FDDN_arr,V_true_LTR_arr,break_flag_arr,c_fs = network.train(X_train,dia_train,batch_idx,MTR_DuctArea,target_df,zeta_col_names,V_max_org)
    # Printing out the training progress
    MSE_flowloss = MSE(V_hat_FDDN_arr,V_true_LTR_arr) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... MSE Flowloss: " + str(round(MSE_flowloss,4))[:5] + " ... Error Function loss: " + str(round(error,4)))
    print('... Training Iteration:',ii)       
    sys.stdout.flush()
    MSE_flowloss_hist.append(MSE_flowloss)
    err_loss_hist.append(error)
    early_stopping_bf = network.early_stopping(MSE_flowloss_hist, 10) # Call early stopping function to monitor last 10 epochs of the loss array
    if (1 in break_flag_arr): # To remove converged HoV 
        if(0 in list(batch_idx)):
            row_ids = [0] # Manually include [0] index if it is in the current batch
        else:
            row_ids = list(filter(lambda e: e != 0, list(batch_idx * np.asarray(break_flag_arr)))) #Multiply row_ID list with bf_list and remove 0s
        HoVs = target_df[['HoV']].loc[row_ids].values.tolist()        
        final_cfs = list(filter(lambda e: e != 0, list(c_fs * np.asarray(break_flag_arr)))) #Multiply MTR_cf_array with bf_list and remove 0s
        V_LTR_final = list(filter(lambda e: e != 0, list(V_true_LTR_arr * np.asarray(break_flag_arr))))
        V_FDDN_final = list(filter(lambda e: e != 0, list(V_hat_FDDN_arr * np.asarray(break_flag_arr))))
        print('REMOVING CONVERGED HoVs:',HoVs,'Row Idx Dropped:',row_ids)
        print("NN_CF_Array:", c_fs)
        print("Final CFs to Append:", final_cfs)
        for hov, cf, V_LTR, V_FDDN in zip(HoVs, final_cfs, V_LTR_final, V_FDDN_final):
            c_f_hist['HoV'].append(hov[0])
            c_f_hist['c_f_hat'].append(cf)
            c_f_hist['ConvergedEpoch'].append(ii)
            c_f_hist['FlowRate_Diff_(LTR-FDDN)'].append(V_LTR - V_FDDN )
        train_data_idx = list(set(train_data_idx).difference(set(row_ids)))
        if (len(train_data_idx) < batch_sample_size): # reduce batch size when available data is less than batch size
            batch_sample_size = batch_sample_size - len(row_ids)        
    if (len(train_data_idx) == 0) or (ii == iterations - 1) or (early_stopping_bf == 1):
        sys.stdout.write('TRAINING PROCESS COMPLETED -')
        if (len(train_data_idx) == 0):
            print(' All HoVs Converged')
        elif (ii == iterations - 1):
            print(' Set Nr. of Training Epochs Reached')
        elif (early_stopping_bf == 1):
            print("EARLY STOPPING ACTIVATED")
        break


EPOCH 0 - Length of Training Dataset: 34
Current Indices in Batch: [33  4 31]

NN output (C_f): 0.009020272916633987
New_Zeta: 0.03827277285199809
FDDN Solver Output: [[101 48 152 99.3 52.4 162 15.9 34.2 34.2 34.3 33.5 33.1 15.5 33.7 34 33.4 33 33]]
New_Zeta_epsi: 0.08776652568672252
FDDN Solver Output: [[100 47.8 151 98.8 52.2 161 15.8 34.1 34 34.1 33.4 32.9 15.4 33.5 33.8 33.3 32.9 32.9]]
Row ID: 33
HoV: ['T2']
Flow Rate Difference (LTR - FDDN): -240.47563815285582 l/s

NN output (C_f): 0.006112249300556649
New_Zeta: 0.02593410767689758
FDDN Solver Output: [[102 48.9 155 101 53.6 165 16.1 34.7 34.7 34.8 34 33.6 15.7 34.1 34.4 33.9 33.4 33.5]]
New_Zeta_epsi: 0.07542786051162202
FDDN Solver Output: [[102 48.7 154 101 53.3 165 16 34.6 34.5 34.6 33.9 33.4 15.6 34 34.3 33.7 33.3 33.3]]
Row ID: 4
HoV: ['A5']
Flow Rate Difference (LTR - FDDN): -257.32417489647776 l/s

NN output is -ve. Taking |(c_f)| value: [0.182]
New_Zeta: 0.4419824914510516
FDDN Solver Output: [[97.8 46.7 148 94.8 50.1 


EPOCH 6 - Length of Training Dataset: 34
Current Indices in Batch: [16  3 21]

NN output (C_f): 1.4837180434393735
New_Zeta: 6.887476038486105
FDDN Solver Output: [[66.9 31.9 101 65.9 34.8 107 10.6 22.8 22.7 22.8 22.3 22 10.3 22.4 22.6 22.3 22 22]]
New_Zeta_epsi: 6.941624841131269
FDDN Solver Output: [[66.7 31.8 101 65.8 34.8 107 10.5 22.7 22.7 22.7 22.3 22 10.3 22.4 22.6 22.2 21.9 21.9]]
Row ID: 16
HoV: ['F1']
Flow Rate Difference (LTR - FDDN): 68.96529738848028 l/s

NN output (C_f): 1.3078678006689703
New_Zeta: 4.220767435099389
FDDN Solver Output: [[74.9 35.7 113 73.9 39 120 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi: 4.258412470548896
FDDN Solver Output: [[74.8 35.7 113 73.7 38.9 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
Row ID: 3
HoV: ['A4']
Flow Rate Difference (LTR - FDDN): 1.2139503701090462 l/s
Percentage Diff between flowrates 0.16595252631057245 %

NN output (C_f): 1.41495431597796
New_Zeta: 6.003612710155895
FDDN Solve

FDDN Solver Output: [[84.7 40.4 128 82.1 43.4 134 13.3 28.8 28.8 28.8 28.2 27.8 12.8 27.9 28.1 27.7 27.3 27.3]]
New_Zeta_epsi: 2.4195007496912337
FDDN Solver Output: [[84.5 40.3 127 81.9 43.3 133 13.3 28.8 28.7 28.8 28.2 27.8 12.8 27.8 28.1 27.6 27.3 27.3]]
Row ID: 31
HoV: ['S5']
Flow Rate Difference (LTR - FDDN): -60.85994931331868 l/s
No. of Records: 3
Progress: 4.4% ... MSE Flowloss: 1841. ... Error Function loss: 0.0102... Training Iteration: 11

EPOCH 12 - Length of Training Dataset: 32
Current Indices in Batch: [ 0 14 30]

NN output (C_f): 1.2615018779041856
New_Zeta: 5.352518185604141
FDDN Solver Output: [[72 34.4 109 71 37.5 116 11.4 24.5 24.5 24.5 24 23.7 11.1 24.2 24.4 24 23.7 23.7]]
New_Zeta_epsi: 5.402011938438865
FDDN Solver Output: [[71.9 34.3 108 70.9 37.4 115 11.3 24.5 24.4 24.5 24 23.6 11.1 24.1 24.3 23.9 23.6 23.6]]
Row ID: 0
HoV: ['A1']
Flow Rate Difference (LTR - FDDN): 35.219237155983365 l/s

NN output (C_f): 1.259490240920018
New_Zeta: 5.343982864548194
FDDN Solve

FDDN Solver Output: [[76.2 36.4 115 75.2 39.7 122 12.2 26.3 26.2 26.3 25.7 25.4 11.7 25.6 25.8 25.4 25 25.1]]
Row ID: 10
HoV: ['C5']
Flow Rate Difference (LTR - FDDN): -19.812651994217845 l/s

NN output (C_f): 1.240985315510076
New_Zeta: 5.76070140986866
FDDN Solver Output: [[70 33.4 106 69 36.5 112 11 23.8 23.8 23.9 23.3 23 10.8 23.5 23.7 23.3 23 23]]
New_Zeta_epsi: 5.814850212513823
FDDN Solver Output: [[69.8 33.3 105 68.9 36.4 112 11 23.8 23.7 23.8 23.3 23 10.8 23.4 23.6 23.2 22.9 23]]
Row ID: 17
HoV: ['F2']
Flow Rate Difference (LTR - FDDN): 38.391540278115485 l/s
No. of Records: 3
Progress: 6.8% ... MSE Flowloss: 1232. ... Error Function loss: 0.0197... Training Iteration: 17

EPOCH 18 - Length of Training Dataset: 31
Current Indices in Batch: [10 25  7]

NN output (C_f): 1.1485418849673295
New_Zeta: 4.063686171985528
FDDN Solver Output: [[76 36.3 115 75 39.6 122 12.1 26.2 26.1 26.2 25.7 25.3 11.7 25.5 25.7 25.3 25 25]]
New_Zeta_epsi: 4.10495797656475
FDDN Solver Output: [[75.9 36

FDDN Solver Output: [[77 36.8 116 76 40.1 124 12.2 26.2 26.2 26.2 25.7 25.3 11.9 25.8 26 25.6 25.3 25.3]]
New_Zeta_epsi: 3.9966117678230058
FDDN Solver Output: [[76.8 36.7 116 75.8 40 123 12.1 26.2 26.1 26.2 25.6 25.3 11.8 25.8 26 25.6 25.2 25.2]]
Row ID: 1
HoV: ['A2']
Flow Rate Difference (LTR - FDDN): -13.76806932733632 l/s

NN output (C_f): 0.9498266062510559
New_Zeta: 4.030088478009845
FDDN Solver Output: [[76.5 36.5 115 75.4 39.8 123 12.1 26 26 26.1 25.5 25.2 11.8 25.7 25.9 25.4 25.1 25.1]]
New_Zeta_epsi: 4.0795822308445695
FDDN Solver Output: [[76.3 36.4 115 75.2 39.7 123 12 26 25.9 26 25.4 25.1 11.8 25.6 25.8 25.4 25.1 25.1]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -75.58651229024326 l/s
No. of Records: 3
Progress: 9.2% ... MSE Flowloss: 1967. ... Error Function loss: 0.0056... Training Iteration: 23
REMOVING CONVERGED HoVs: [['T2']] Row Idx Dropped: [33]
NN_CF_Array: [0.9919451935527354, 0.9737874402879785, 0.9498266062510559]
Final CFs to Append: [0.991945193

...

FDDN Solver Output: [[68.3 32.6 103 67.4 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.5 22.5]]
New_Zeta_epsi: 6.360069074523966
FDDN Solver Output: [[68.2 32.5 103 67.2 35.5 110 10.8 23.2 23.2 23.2 22.8 22.4 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: 5
HoV: ['A6']
Flow Rate Difference (LTR - FDDN): 76.62913554071008 l/s
No. of Records: 3
Progress: 79.6% ... MSE Flowloss: 4427. ... Error Function loss: 0.1268... Training Iteration: 199

EPOCH 200 - Length of Training Dataset: 5
Current Indices in Batch: [ 5 31 16]

NN output (C_f): 1.109971015054009
New_Zeta: 4.709576852505736
FDDN Solver Output: [[73.5 35.1 111 72.5 38.3 118 11.6 25 25 25.1 24.5 24.2 11.3 24.7 24.9 24.5 24.1 24.2]]
New_Zeta_epsi: 4.759070605340462
FDDN Solver Output: [[73.3 35 111 72.3 38.2 118 11.6 25 24.9 25 24.5 24.1 11.3 24.6 24.8 24.4 24.1 24.1]]
Row ID: 5
HoV: ['A6']
Flow Rate Difference (LTR - FDDN): 26.123591869618053 l/s

NN output (C_f): 0.9715644405972271
New_Zeta: 2.3652347526978676
FDDN Solve

FDDN Solver Output: [[83.5 39.8 126 80.9 42.8 132 13.2 28.4 28.4 28.4 27.8 27.5 12.6 27.5 27.7 27.3 26.9 27]]
Row ID: 31
HoV: ['S5']
Flow Rate Difference (LTR - FDDN): -51.04502428291153 l/s

NN output (C_f): 1.2239186532703508
New_Zeta: 5.681477309472494
FDDN Solver Output: [[70.4 33.6 106 69.4 36.7 113 11.1 24 23.9 24 23.5 23.2 10.9 23.6 23.8 23.4 23.1 23.1]]
New_Zeta_epsi: 5.735626112117657
FDDN Solver Output: [[70.2 33.5 106 69.2 36.6 113 11.1 23.9 23.9 23.9 23.4 23.1 10.8 23.6 23.7 23.4 23.1 23.1]]
Row ID: 16
HoV: ['F1']
Flow Rate Difference (LTR - FDDN): 34.577428082399365 l/s
No. of Records: 3
Progress: 82.0% ... MSE Flowloss: 1798. ... Error Function loss: 0.0206... Training Iteration: 205

EPOCH 206 - Length of Training Dataset: 4
Current Indices in Batch: [31 20 16]

NN output (C_f): 0.8658115257679035
New_Zeta: 2.107783513334211
FDDN Solver Output: [[86.2 41.2 130 83.6 44.2 136 13.6 29.4 29.3 29.4 28.7 28.4 13.1 28.4 28.6 28.2 27.8 27.9]]
New_Zeta_epsi: 2.136181187732894
FDD


EPOCH 212 - Length of Training Dataset: 2
Current Indices in Batch: [ 5 31]

NN output (C_f): 1.3729332482376533
New_Zeta: 5.825318461690592
FDDN Solver Output: [[69.8 33.3 105 68.8 36.4 112 11 23.8 23.7 23.8 23.3 23 10.8 23.4 23.6 23.2 22.9 22.9]]
New_Zeta_epsi: 5.874812214525317
FDDN Solver Output: [[69.6 33.2 105 68.7 36.3 112 11 23.7 23.7 23.7 23.2 22.9 10.7 23.4 23.6 23.2 22.9 22.9]]
Row ID: 5
HoV: ['A6']
Flow Rate Difference (LTR - FDDN): 62.453164405772554 l/s

NN output (C_f): 1.2179030844462746
New_Zeta: 2.9649363237085065
FDDN Solver Output: [[81.7 39 123 79.2 41.8 129 12.9 27.8 27.8 27.8 27.2 26.9 12.4 26.9 27.1 26.7 26.4 26.4]]
New_Zeta_epsi: 2.9933339981071896
FDDN Solver Output: [[81.5 38.9 123 79.1 41.8 129 12.9 27.8 27.7 27.8 27.2 26.8 12.3 26.9 27.1 26.7 26.3 26.3]]
Row ID: 31
HoV: ['S5']
Flow Rate Difference (LTR - FDDN): -32.23918093175814 l/s
No. of Records: 2
Progress: 84.8% ... MSE Flowloss: 2469. ... Error Function loss: 0.0509... Training Iteration: 212

EPOCH 


EPOCH 223 - Length of Training Dataset: 1
Current Indices in Batch: [31]

NN output (C_f): 1.4815147413088183
New_Zeta: 3.6066883537069305
FDDN Solver Output: [[78.7 37.6 119 76.3 40.3 124 12.4 26.8 26.8 26.8 26.2 25.9 11.9 25.9 26.2 25.7 25.4 25.4]]
New_Zeta_epsi: 3.6350860281056137
FDDN Solver Output: [[78.6 37.5 118 76.2 40.2 124 12.4 26.8 26.7 26.8 26.2 25.8 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 31
HoV: ['S5']
Flow Rate Difference (LTR - FDDN): -3.536739029079399 l/s
No. of Records: 1
Progress: 89.2% ... MSE Flowloss: 12.50 ... Error Function loss: 0.1159... Training Iteration: 223

EPOCH 224 - Length of Training Dataset: 1
Current Indices in Batch: [31]

NN output (C_f): 1.4814297660033526
New_Zeta: 3.60648148472613
FDDN Solver Output: [[78.7 37.6 119 76.3 40.3 124 12.4 26.8 26.8 26.8 26.2 25.9 11.9 25.9 26.2 25.7 25.4 25.4]]
New_Zeta_epsi: 3.6348791591248126
FDDN Solver Output: [[78.6 37.5 118 76.2 40.2 124 12.4 26.8 26.7 26.8 26.2 25.8 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 


EPOCH 237 - Length of Training Dataset: 1
Current Indices in Batch: [31]

NN output (C_f): 1.4815165758782827
New_Zeta: 3.6066928198928796
FDDN Solver Output: [[78.7 37.6 119 76.3 40.3 124 12.4 26.8 26.8 26.8 26.2 25.9 11.9 25.9 26.2 25.7 25.4 25.4]]
New_Zeta_epsi: 3.6350904942915627
FDDN Solver Output: [[78.6 37.5 118 76.2 40.2 124 12.4 26.8 26.7 26.8 26.2 25.8 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 31
HoV: ['S5']
Flow Rate Difference (LTR - FDDN): -3.536740567636457 l/s
No. of Records: 1
Progress: 94.8% ... MSE Flowloss: 12.50 ... Error Function loss: 0.1159... Training Iteration: 237

EPOCH 238 - Length of Training Dataset: 1
Current Indices in Batch: [31]

NN output (C_f): 1.4814315146187904
New_Zeta: 3.606485741660441
FDDN Solver Output: [[78.7 37.6 119 76.3 40.3 124 12.4 26.8 26.8 26.8 26.2 25.9 11.9 25.9 26.2 25.7 25.4 25.4]]
New_Zeta_epsi: 3.634883416059124
FDDN Solver Output: [[78.6 37.5 118 76.2 40.2 124 12.4 26.8 26.7 26.8 26.2 25.8 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 

In [25]:
print("MSE_flowloss_history:",MSE_flowloss_hist)

MSE_flowloss_history: [53123.131300857436, 146206.50633195715, 98662.68741813523, 88399.50107676984, 40072.30619456263, 8977.148363242364, 1863.5211704464352, 262.69429617132005, 1462.2299721592915, 1984.3230759646442, 749.7896043499491, 1841.9073197558102, 944.8476230753558, 317.72017676176216, 161.6474579094705, 643.6240174659415, 1643.7670075552858, 1232.9058287413618, 519.94702712217, 2274.527622600726, 2606.92791441046, 1228.963176120711, 609.1167988968668, 1967.8172722761267, 677.3338735114386, 176.07548639277948, 59.682463066944514, 1690.3402082697332, 3001.9862471548217, 789.1752014250491, 487.3829101605258, 800.8126548213686, 690.4034732094425, 1238.4878919263895, 1865.176506924141, 3193.9979195255505, 1391.2839558230462, 510.5870202770216, 606.4530484618688, 374.6169300975041, 1800.6654628724634, 1271.883832891294, 1770.0967578739164, 166.4166717507358, 54.915199171808105, 1137.254483639701, 341.59596496996534, 428.2798339545487, 1341.1989305632044, 187.46133617885448, 308.26

In [26]:
print("Error_loss_history:",err_loss_hist)

Error_loss_history: [0.5764646846820525, 38.49065271980641, 19.666031023326795, 9.93727344488475, 2.998373413583409, 0.5942548633778963, 0.08828091456584597, 0.0017815257415667817, 0.006389181788539985, 0.08984686767162582, 0.007986730274485127, 0.01022877742507947, 0.03290596389495102, 0.001564443909650701, 0.0066181224981670195, 0.0033056914993835817, 0.00763335259199069, 0.01966991456830082, 0.016805057651432517, 0.06921298048035197, 0.008212188999258322, 0.08006552388853767, 0.01948943324773015, 0.0056016850689185466, 0.04535507499506745, 0.004953422393959074, 0.0008796395027263477, 0.005745982879135381, 0.0771195772752545, 0.006849948312545313, 0.009494430901595062, 0.007233450086518352, 0.007908859488939128, 0.05697719282601099, 0.007490949501491666, 0.11754666021516472, 0.007138820390900557, 0.025487439869678063, 0.012781698730593153, 0.03566301465785797, 0.041589022186110096, 0.008333653191961304, 0.03424975420429537, 0.0005530198636489374, 0.0007435433155733129, 0.004126592686

In [27]:
cf_df = pd.DataFrame(data=c_f_hist)
cf_df.to_csv('data_output/NN_1CF_Output_miniBatch.csv',index=False)
cf_df

,HoV,c_f_hat,FlowRate_Diff_(LTR-FDDN),ConvergedEpoch
0,A4,1.307868,1.213950,6
1,S3,1.021167,1.813756,7
2,A5,0.999159,-0.593881,16
3,C5,1.272813,0.290181,19
4,T2,0.991945,0.755807,23
5,M2,1.233107,1.234050,24
6,A2,1.072544,1.714065,25
7,C6,1.213337,-0.630215,39
8,S2,0.991574,0.077467,45
9,F2,0.998760,1.430830,53


### Plot Training Loss

In [28]:
hover1    = [('Epoch', '@x'),('MSE FlowLoss', '@y') ]
p1 = figure(width=1600, tooltips=hover1)
p1.line(list(range(1, len(MSE_flowloss_hist)+1)), MSE_flowloss_hist, line_width=2, color='blue', alpha=0.8)
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
show(p1)

In [29]:
hover2    = [('Epoch', '@x'),('Error', '@y') ]
p2 = figure(width=1600, tooltips=hover2)
p2.line(list(range(1, len(err_loss_hist)+1)), err_loss_hist, line_width=2, color='red', alpha=0.8)
p2.yaxis.axis_label = "Error (Loss Function)"
p2.xaxis.axis_label = "Epochs"
show(p2)